![alt text](Report_images/Robo-Geek_logo_Canada_150.PNG)

# STEM Club Project -Fall 2017

## Acknowledgments and Inspiration:

Robo-Geek is very proud to present the Self Driving Car Simulator Project with STEM club. This couldn't be possible without the great opportunity to work with the open source community. 

Udacity open-sourced a self driving car simulator designed for Self Driving Car Nano Degree program.
![alt text](Report_images/Udacity_self_driving_car_program.PNG)

The objective of the project is to use the self driving car simulator to train a neural network by collecting images as we run and then use the model to run the simulator autonomously.

Searching up for inspiration how to implement this project in Python, we run into this fantastic video from YouTuber star Siraj Raval with a github project that we used as the base of the STEM club project:

![alt text](Report_images/Siraj_Raval.JPG)

https://www.youtube.com/watch?v=EaY5QiZwSP4

Again we are very thankful for the open source community, our project is shared at:

https://github.com/robogeekcanada/data


## High Level Understanding of the Project

In this project we will use the Udacity simulator to conduct supervised training meaning we will drive the car and record our movements and collect the frames of the cameras inside the simulator. The images collected will be fed to a convolutional neural network, in addition augmentation of the images to simulate different conditions will be created and fed to the CNN. The design of the CNN is based on Nvidia's self driving car paper:

https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/

Finally with a trained CNN, we can drive the car autonomously. The CNN will decide on acceleration and steering. There are three Python programs that make this possible: model.py, utils. py and drive.py


![alt text](Report_images/High_level_concept.JPG)

Nine students ages 11-15 participated in this Robo-Geek project. They met weekly for 1.5 hours for 10 weeks. The project was divided in 2 parts:

#### Part 1: Data adquisition with Udacity Simulator + Model training

#### Part 2: Testing model with AWS servers and improvements to drive.py


## Part 1: Data Adquisition with Udacity Simulator and Model Training


### Udacity Simulator

Udacity simulator has two modes: Training Mode and Autonomous mode:

![alt text](Report_images/Udacity_simulator.JPG)


The simulator world is built in Unity and new tracks can be added or changes to the physics as well. We only focused on the Lake race-track, however the same steps could be used to train on other tracks. Unity is open source free 3D modelling software, for more information https://unity3d.com/


![alt text](Report_images/Unity_race_tracks.PNG)



The code for the simulator can be found at:
https://github.com/udacity/self-driving-car-sim


### 1.1 Data Adquisition in Simulator Training Mode:

The Udacity simulator can be configured and two race tracks are available. In this project we only train in the lake track.

Once the simulator is opened under training mode, we can simply go around the track using keyboard, mouse or joystick.
There is the option to record the information by pressing the Red Record button.

![alt text](Report_images/Simulator_training.JPG)

Three images are recorded per frame at different angles: LEFT, RIGHT and CENTER in a data folder and the log of where the images are stored is saved under driving_log.CSV

![alt text](Report_images/data_folder.JPG)

#### Left image:

![alt text](Report_images/left.PNG)  

#### Right image:

![alt text](Report_images/right.PNG)

#### Center image:

![alt text](Report_images/center.PNG)

#### Nvidia's Hardware design 

As per Nvidia's hardware design shown below:

![alt text](Report_images/Nvidia_hardware_model.PNG)

Records images from center, left, and right cameras w/ associated steering angle, speed, throttle and brake saves to driving_log.CSV.

![alt text](Report_images/driving_log_csv.JPG)




### 1.2 Model training using CNN-  Behavioral cloning


The design and optimization of a convolutional network is outside the scope of this project. As mentioned previously, the project was inspired from Siraj Raval's project posted in who created a wrapper from the original project Naoki Shibuya:

https://github.com/llSourcell/How_to_simulate_a_self_driving_car

The 9 layer convolutional neural network consists of a 1 normalization layer, 5 convolutional layers and 3 connected layers that lead to output control vehicle as shown in the picture below:


![alt text](Report_images/cnn_architecture.PNG)



In [ ]:
def build_model(args):
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(args.keep_prob))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model

Due to the high amount of computation, a powerful computer is required. A single CPU may take several years to train the model. We decided to test different servers in Google Cloud and AWS. We decided to stick with AWS Windows Server, there is a server per every 2 students. Due to cost management we only turn them on when required. Ideally each student will have their own.


![alt text](Report_images/AWS_Robo-Geek_servers.JPG)

To connect to the servers we use Remote Desktop Connection, using the assigned user accounts:
Anaconda is pre-installed for each user. And a copy of the project folder is stored in the Documents folder.

![alt text](Report_images/Remote_Desktop_Connection.JPG)

In this project we didn't cover how to install the environment for this project. More information can be found from the original project. Due to time constraint we preferred not to get into troubleshooting of installation. Let's just say it's not a smooth process all around.

Using Conda Prompt, we execute the following commands in the project folder:

![alt text](Report_images/training_model_conda.PNG)

In [ ]:
python model.py

Depending on the speed of the server training may take between an hour to a few hours. With the servers we selected it took 1 hour. The output of is model-<epoch>.h5 file that we will use to run in Autonomous Mode.

In order to undestand what happens in CNN while it's training, there is an excellent paper by Naoki Shibuya:

https://github.com/naokishibuya/car-behavioral-cloning

It shows what happens in the Image augmentation process. The code that makes this happen is shown below under utils.py


In [ ]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers

The augment function in utils.py calls the other functions that manipulate the images including random_flips, random_translates, random_shadow and random_brightness. As per the Nvidia's paper: "we augment the data by adding artificial shifts and rotations to teach the network how to recover from a poor position or orientation. The magnitidue of these perturbations is chosen randomly from a normal distribution...Artificially augmenting the data does add undesirable artifacts as the magnitude increases."

In [ ]:
def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    """
    Generate an augumented image and adjust steering angle.
    (The steering angle is associated with the center image)
    """
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = random_shadow(image)
    image = random_brightness(image)
    return image, steering_angle

<video controls src="Report_videos/cnn training visualization.mp4" />

## Part 2: Testing Model with AWS server and improving drive.py

The processing speed required is significant, GPUs are recommended. Since we didn't have them available for all students, we deciced to use AWS servers. 


### 2.1 Testing the model with AWS

All students were given an account to connect to an AWS server. We tested the following servers to see the impact of computing processing:

![alt text](Report_images/AWS_servers.JPG)


### 2.2 Improving drive.py

We made small modifications to the existing drive.py to control the speed and augment the signal. We did not have time to balance the data. Please refer to final code drive5.py

In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model

import utils

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None

MAX_SPEED = 10
MIN_SPEED = 5

speed_limit = MAX_SPEED

Adjustment_factor = 1.2
Speed_adjustment =0.8

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = float(data["steering_angle"])
        # The current throttle of the car
        throttle = float(data["throttle"])
        # The current speed of the car
        speed = float(data["speed"])*Speed_adjustment
        # The current image from the center camera of the car
        image = Image.open(BytesIO(base64.b64decode(data["image"])))
        try:
            image = np.asarray(image)       # from PIL image to numpy array
            image = utils.preprocess(image) # apply the preprocessing
            image = np.array([image])       # the model expects 4D array

            # predict the steering angle for the image
            steering_angle = float(model.predict(image, batch_size=1))*Adjustment_factor
            # lower the throttle as the speed increases
            # if the speed is above the current speed limit, we are on a downhill.
            # make sure we slow down first and then go back to the original max speed.

            if (steering_angle*steering_angle) > 0.04:
                steering_angle = steering_angle*0.8

                print("tight corner...")
    

            global speed_limit
            if speed > speed_limit:
                speed_limit = MIN_SPEED  # slow down

            else:
                speed_limit = MAX_SPEED
            #throttle = 1.0 - steering_angle**2 - (speed/speed_limit)**2
            throttle = 0.18

            print('{} {} {}'.format(steering_angle, throttle, speed))
            send_control(steering_angle, throttle)
        except Exception as e:
            print(e)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )
    parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )
    args = parser.parse_args()

    model = load_model(args.model)

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)



## Testing mode

With m4.x16large we run drive5.py in autonomous mode and success a complete lap!


![alt text](https://cdn-images-1.medium.com/max/1440/1*nlusa_fC5BnsgnWPFnov7Q.tiff "Logo Title Text 1")

### Further reading:

Thanks to the wonderful open source community that is actively improving on this challenge, there are many other projects that beg to be reviewed that were not covered due to time constraint. These projects includes further analysis on data pre-processing, PID loops, discussion on best training techniques, not to mention other ideas to improve driving and predictions. Below a list of github projects that we used as references:

https://github.com/Michael-Tu/Udacity-Self-Driving-Car/tree/master/p3-behavioral-cloning

https://github.com/yazeedalrubyli/SDCND/tree/master/Term1/Behavioral-Cloning

https://github.com/sibyjackgrove/Car_driving_Behaviour_cloning

https://github.com/morganel/self-driving-cars/tree/master/behavioral-cloning

